# Counting the Prameters and Operations for MicroNet

Here is the plan.

You can select the version of micronet. There are two versions and the checkpoints we uploaded on github are from ver2 network,
so if you want to verify our model, please choose the network version as 'ver2'.

Our network was trained with FP-32 precision, but we will calculate our score with freebie quantization rule.
Therefore, we calculate the number of parameters and multiplication operations with the assumption of 16bit not on addition.

Our network has below operations:

- Module: `Convolution`, `AveragePool`, `FullyConnected`
- Activation: `HardSwish`, `Sigmoid`

We also used the batch normalization, but we heard that we can count except BN in terms of storage and for math operations even BN is not merged just as the code shows.

***
We calculate the math operations with our method from `Counting` as considering the sparsity.
we count the `Hard Swish` function as below:

addition: 1, multiplication: 4

Hard swish function have this formula: x * relu_6(x + 3.0) / 6.0. From your previous annoucnments,
relu_6 function have two multiplicaiton operations, so you can easily get above results.

In our counting code, actually this doesnot capture our hard swish fuction, but count with relu 6 function.
However, we only use the relu6 function in Hard swish activation, so it is proper to get exact flops count.

In [1]:
import os
import time
import argparse
import shutil
import math

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from Counting import count
import matplotlib.pyplot as plt

from Utils import *
from Models import *

### These are for Score Calculation.
- count_nonzero: count the number of nonzero parameters except BN
- micro_score: get score. Flops are already calculated with sparsity, so we multiply the non zero ratio only at params. In addition, we did not use any quantization so that we can assume freebie quantization rule. Therefore, we calculate the score with multiplying 0.5 both on multiplications and params.

In [2]:
def count_nonzero(net):
    num = 0 
    for module in net.parameters():
        if module.ndimension() != 1:
            num += torch.sum(torch.abs(module.flatten()) != 0.)
    return num.item()

In [3]:
def micro_score(net, precision = 'Freebie'):
    input = torch.randn(1, 3, 32, 32).to(net.device)
    addflops, multflops, params = count(net, inputs=(input, ))
    non_zero_ratio = count_nonzero(net) / params

    params = params * non_zero_ratio
    #use fp-16bit
    if precision == 'Freebie':
        multflops = multflops / 2
        params = params / 2
    
    score = (params/36500000 + (addflops + multflops)/10490000000)
    print('Non zero ratio: {}'.format(non_zero_ratio))
    print('Score: {}, flops: {}, params: {}'.format(score, addflops + multflops, params))
    return score

### Build Model

To verify our checkpoints, here is the details of checkpoints.
Here, we will verify our ver2 checkpoints. They are in the `Checkpoint_ver2` folder.

- micronet_v2.t7: version 2 model with no pruning
- micronet_v2_pr_15.t7: version 2 model with 15% pruning
- micronet_v2_pr_30.t7: version 2 model with 30% pruning
- micronet_v2_pr_45.t7: version 2 model with 45% pruning
- micronet_v2_pr_final.t7: version 2 model with 65% pruning

The `micronet_v2.t7` and `micronet_v2_pr_final.t7` files have four checkpoints with dictionary type, so if you want to load the checkpoints for verification, load the checkpoints with the key <strong>'net4'.

In [4]:
net = MicroNet(ver = 'ver2', num_classes = 100, add_se = True, Activation = 'HSwish')
train_loader, test_loader, num_classes = transform_data_set('CIFAR100', batch_size = 128, augmentation = 'FastAuto')

### Evaluate the test accuracy
We here load the final pruned network with 65%, and evaluate with <strong>CIFAR100 test dataset.

In [15]:
checkpoint = torch.load('./Checkpoint_ver2/micronet_v2.t7')
net.load_state_dict(checkpoint['net2'], strict = False)
net.to(net.device)
eval_32bit(net, test_loader)

100%|██████████| 100/100 [00:02<00:00, 42.92it/s]

Loss: 0.714 | Acc1: 80.780% | Acc5: 95.980%


(80.78, 0.7141554743051529)

### Score
Then, you can get the score.

In [9]:
score = micro_score(net)

Count has not implemented counting method for  CrossEntropyLoss()
Register FLOP counter for module Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(32, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module Conv2d(32, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(80, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module Conv2d(80, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=80, bias=False)
Register FLOP counter for module BatchNorm2d(80, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module Conv2d(80, 20, kernel_size=(1, 1), stride=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(20, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU6(inplace)
Register